In [154]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt  # to plot
import matplotlib as mpl

from hardwareModel import HardwareModel
import networkx as nx

from scipy import optimize       # to compare

import seaborn as sns
sns.set(context="talk", style="darkgrid", palette="hls", font="sans-serif", font_scale=1.05)

FIGSIZE = (19, 8)  #: Figure size, in inches!
mpl.rcParams['figure.figsize'] = FIGSIZE

In [177]:
def annealing(random_start,  # get rid of function parameters
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              positions,
              edges,
              areas,
              maxsteps=1000,
              debug=True):
    """ Optimize the black-box function 'cost_function' with the simulated annealing algorithm."""
    for i in range(len(positions)):
        state = random_start()
        positions[i] = state
    
    wireLengths = wirelengths(positions, edges)
    cost = cost_function(wireLengths, positions, areas)
    states, costs = [state], [cost]
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)

        for i in range(len(positions)):
            new_state = random_neighbour(positions[i], fraction)
            positions[i] = new_state
            wireLengths = wirelengths(positions, edges)
            new_cost = cost_function(wireLengths, positions, areas)
            #if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
            if acceptance_probability(cost, new_cost, T) > rn.random():
                state, cost = new_state, new_cost
                states.append(state)
                costs.append(cost)
            # print("  ==> Accept it!")
        # else:
        #    print("  ==> Reject it...")
        
        
    return state, cost_function(wireLengths, positions, areas), states, costs

In [178]:
hw = HardwareModel(cfg='aladdin_const_with_mem')
digraph = hw.netlist
N = len(digraph.nodes)

netlist: [('Add0', {'type': 'pe', 'function': 'Add', 'in_use': 0, 'idx': 0}), ('Regs0', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 0}), ('Regs1', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 1}), ('Regs2', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 2}), ('Mult0', {'type': 'pe', 'function': 'Mult', 'in_use': 0, 'idx': 0}), ('Eq0', {'type': 'pe', 'function': 'Eq', 'in_use': 0, 'idx': 0}), ('Buf0', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 0, 'size': 22}), ('Buf1', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 1, 'size': 22}), ('Mem0', {'type': 'mem', 'function': 'MainMem', 'in_use': 0, 'idx': 0, 'size': 1024}), ('Regs3', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 3})]


In [179]:
interval = (-10, 10)
N = 3
#wirelengths = {(0, 1) : 2, (2, 3) : 2}
positions = [(0, 0), (0, 0), (2, 0)]
edges = [(0, 1), (0, 2)]
areas = [1, 1, 1]

def wirelengths(x, e):
    repeats = set()
    output = {}

    for edge in e:
        if (edge[1], edge[0]) not in repeats:
            repeats.add(edge)
            position0 = x[edge[0]]
            position1 = x[edge[1]]
            wirelength = ((position0[0] - position1[0]) ** 2 + (position0[1] - position1[1]) ** 2) ** 0.5
            output[edge] = wirelength

    return output

def f(x, positions, areas):
    """ Function to minimize."""
    #print(areas)
    return sum(x.values()) + overlapCost(positions, areas)

def overlap(first, sec, area1, area2):
    s1 = (area1 ** 0.5) / 2
    s2 = (area2 ** 0.5) / 2
    deltaX = min((first[0] + s1, sec[0] + s2)) + max((first[0] - s1, sec[0] - s2))
    deltaY = min((first[1] + s1, sec[1] + s2)) + max((first[1] - s1, sec[1] - s2))
    return deltaX * deltaY

def overlapCost(pVectors, areas, const=1):
    totalCost = 0
    #print(areas)
    
    for i in range(len(pVectors) - 1):
        for j in range(i, len(pVectors)):
            totalCost += overlap(pVectors[i], pVectors[j], areas[i], areas[j])

    return totalCost

def clip(x):
    """ Force x to be in the interval."""
    a, b = interval
    return max(min(x, b), a)

In [180]:
def random_start():
    """ Random point in the interval."""
    a, b = interval
    x = a + (b - a) * rn.random_sample()
    y = a + (b - a) * rn.random_sample()
    return (x, y)

def init_positions():
    return random.Generator.normal(loc=0.0, scale=1.0, size=None)

In [181]:
def cost_function(x, positions, areas):
    """ Cost of x = f(x)."""
    #print(areas)
    return f(x, positions, areas)

In [182]:
def random_neighbour(x, fraction=1):
    """Move a little bit x, from the left or the right."""
    amplitude = (max(interval) - min(interval)) * (1 - fraction) / 10
    deltaX = (-amplitude/2.) + amplitude * rn.random_sample()
    deltaY = (-amplitude/2.) + amplitude * rn.random_sample()
    return (clip(x[0] + deltaX), clip(x[1] + deltaY))

In [183]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        # print("    - Acceptance probabilty = 1 as new_cost = {} < cost = {}...".format(new_cost, cost))
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        # print("    - Acceptance probabilty = {:.3g}...".format(p))
        return p

In [184]:
def temperature(fraction):
    """ Example of temperature dicreasing as the process goes on."""
    return max(0.01, min(1, 1 - fraction))

In [185]:
annealing(random_start, cost_function, random_neighbour, acceptance_probability, temperature, positions, edges, areas, maxsteps=30, debug=True);

In [187]:
state, c, states, costs = annealing(random_start, cost_function, random_neighbour, acceptance_probability, temperature, positions, edges, areas, maxsteps=1000, debug=False)

print(positions)

[(-8.978312633279256, 2.0445872797094804), (1.475905625975605, -6.9361781623694885), (-2.9500230640509253, -0.8593229014615302)]
